
# Summary
- 项目目标：预测客户是否会订购银行定期存款产品。
- 数据来源：Kaggle网：https://www.kaggle.com/datasets/adityamhaske/bank-marketing-dataset

# Data
- 导入数据
- 数据基本信息

# EDA
- 特征分布分析
- 类别变量比例
- 数值变量统计和可视化

# Preprocessing
- 缺失值处理
- 类别变量编码
- 数值变量归一化/标准化

# Modeling
- 逻辑回归
- SVM
- 交叉验证

# Eval
- Accuracy / Precision / Recall / F1 / ROC-AUC
- 混淆矩阵
- ROC 曲线

# Conclusion
- 模型表现总结
- 改进建议


In [ ]:
#Data
import pandas as pd
#导入数据
data=pd.read_csv('bank-full.csv')
#查看数据
print(data.head())
print("------------------------------------")
print(data.dtypes)
print("------------------------------------")
print(data.shape)

In [ ]:
#EDA
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------------
# 1️⃣ 类别变量比例分析
# -------------------------------
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('y')  # 排除目标列

for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(data[col].value_counts(normalize=True))  # 比例
    data[col].value_counts().plot(kind='bar', title=col)
    plt.show()

# -------------------------------
# 2️⃣ 数值变量统计和可视化
# -------------------------------
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 数值统计
print("\n数值特征描述统计：")
print(data[numeric_cols].describe())

# 可视化：直方图 + 箱线图
for col in numeric_cols:
    plt.figure(figsize=(12,4))
    
    plt.subplot(1,2,1)
    sns.histplot(data[col], kde=True)
    plt.title(f'{col} Histogram')
    
    plt.subplot(1,2,2)
    sns.boxplot(x=data[col])
    plt.title(f'{col} Boxplot')
    
    plt.show()

In [ ]:
# Preprocessing
#缺失值处理
print(data.isnull().sum())
#数据集中没有缺失值
#编码处理
data["y"]=data["y"].map({"yes":1,"no":0})
data["default"]=data["default"].map({"yes":1,"no":0})
data["loan"]=data["loan"].map({"yes":1,"no":0})
data["housing"]=data["housing"].map({"yes":1,"no":0})
data["marital"]=data["marital"].map({"married":1,"single":0,"divorced":2})
data = pd.concat([data, pd.get_dummies(data["education"], prefix="edu")], axis=1)
data = pd.concat([data, pd.get_dummies(data["job"], prefix="job")], axis=1)#对于特征列较多，可以使用独热编码
data = pd.concat([data, pd.get_dummies(data["contact"], prefix="contact")], axis=1)
data = pd.concat([data, pd.get_dummies(data["month"], prefix="month")], axis=1)
data= pd.concat([data, pd.get_dummies(data["poutcome"], prefix="poutcome")], axis=1)
data.drop(columns=["education","job","contact","month","poutcome"],inplace=True )#删除原始的类别列，只保留编码后的列

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (Linear Kernel)": SVC(kernel="linear", probability=True)
}

scoring = ["accuracy", "f1"]  # 只计算两个指标
results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", model)
    ])
    scores = cross_validate(pipeline, X, y, cv=kf, scoring=scoring, n_jobs=-1)
    results[name] = {metric: scores[f"test_{metric}"].mean() for metric in scoring}

import pandas as pd
df_results = pd.DataFrame(results).T
print(df_results)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# -------------------------------
# 1️⃣ 创建测试集
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# 2️⃣ 遍历模型计算指标并绘图
# -------------------------------
for name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", model)
    ])
    
    # 训练
    pipeline.fit(X_train, y_train)
    
    # 预测
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:,1]  # ROC-AUC用
    
    # 计算指标
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    
    print(f"\n{name} Metrics:")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")
    
    # 混淆矩阵
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"{name} Confusion Matrix")
    plt.show()
    
    # ROC 曲线
    roc_disp = RocCurveDisplay.from_estimator(pipeline, X_test, y_test)
    plt.title(f"{name} ROC Curve")
    plt.show()



Logistic Regression Metrics:
Accuracy: 0.8995, Precision: 0.6564, Recall: 0.3501, F1: 0.4567, ROC-AUC: 0.9045
Logistic Regression 模型表现总结

Accuracy: 0.8995
整体分类准确率较高，说明模型大多数样本预测正确。
Precision: 0.6564
当模型预测客户会订购定期存款时，大约 65.6% 是正确的。说明假阳性（预测订购但实际未订购）还不少。
Recall: 0.3501
模型只捕捉到了 35% 的真实订购客户，漏掉了很多正类（高假阴性）。这是最突出的短板。
F1: 0.4567
F1 较低，表明 Precision 与 Recall 平衡性差，主要是因为 Recall 太低。
ROC-AUC: 0.9045
AUC 很高，说明模型在区分正负类的能力整体很好，但是当前阈值=0.5 的情况下 Recall 偏低。
数据不平衡（y=1 的比例小于 y=0），导致模型更倾向预测负类 → Recall 偏低。
使用了默认决策阈值 0.5，没有针对 Recall 进行优化。
特征可能还需要进一步挖掘，比如交互项或特征选择。

✅ 改进建议

优化决策阈值
调整 Logistic Regression 的预测概率阈值（例如从 0.5 降到 0.3），提升 Recall。
通过 Precision-Recall 曲线 或 Youden’s J statistic 找到更合适的阈值。
类别不平衡处理
在模型中使用 class_weight="balanced"，提升少数类（y=1）的权重。
或者在训练数据上做 上采样（SMOTE）/下采样。
尝试更强模型
树模型（Random Forest / XGBoost / LightGBM）对类别不平衡和非线性特征处理更好，可能比线性模型表现更佳。
特征工程
分析哪些特征对客户订购行为最相关，尝试构造交互特征。
对高基数类别变量做 Target Encoding 或 Embedding，而不是单纯 One-Hot。

📌 总结一句话：模型整体区分能力强（AUC 高），但在 Recall 上严重不足，需要通过 调整阈值 + 类别不平衡处理 来改进，才能真正捕捉到更多的潜在订购客户。

SVM (Linear Kernel) 模型表现总结

SVM (Linear Kernel) Metrics:
Accuracy: 0.8906, Precision: 0.6678, Recall: 0.1861, F1: 0.2910, ROC-AUC: 0.9027

Accuracy: 0.8906
整体准确率依然较高，但略低于 Logistic Regression (0.8995)。
Precsion: 0.6678
当模型预测客户会订购时，大约 66.8% 是正确的，比逻辑回归高一些。假阳性更少。
Recall: 0.1861
召回率非常低，只识别出了不到 20% 的真实订购客户，远低于 Logistic Regression (0.3501)。
→ 说明 SVM 更“保守”，更倾向预测负类。
F1: 0.2910
Precision 较高但 Recall 太低，导致 F1 分数整体很差。
ROC-AUC: 0.9027
AUC 依旧很高，说明整体区分正负类的能力不差，但默认阈值下表现严重偏向负类。

问题诊断
SVM 在线性核下的决策边界偏硬，加上数据不平衡，模型更加不愿意预测少数类（y=1）。
Recall 极低，意味着 SVM 对“真正的潜在客户”几乎抓不到。
默认参数 C=1.0 可能偏向于欠拟合少数类。

✅ 改进建议

调整决策阈值
与 Logistic Regression 类似，可以通过降低阈值提升 Recall。
使用 decision_function / predict_proba 输出分数，再手动设定阈值。
类别不平衡处理
使用 class_weight="balanced" 来增加少数类权重。
或者在数据层面做 SMOTE 上采样 / 欠采样。
调参优化 C 值
增大 C 可能让 SVM 在少数类上更“宽松”，提高 Recall。
尝试非线性模型
RBF Kernel SVM、树模型（Random Forest、XGBoost）可能能更好地捕捉复杂关系。
📌 总结一句话：SVM 在当前设定下过于“保守”，几乎抓不到订购客户。需要通过 类别权重 / 阈值调整 / 调参 来提升 Recall，否则不适合用于实际业务推荐。

🔎 与 Logistic Regression 的对比

Logistic Regression：Recall 高于 SVM，但 Precision 稍低。

SVM：Precision 高于 Logistic Regression，但 Recall 太低。

两者 AUC 都很高 → 模型潜力在，但阈值/类别权重需要调整。